# Imports and Information

Assume you are going to fly into Albany, NY, and then drive to several cities in the NY-New England area to visit with potential client companies. You can’t visit them all; and frankly, some cities may not be worth your time anyway. 

Let’s assume that you have decided that you will drive no more than 1,000 miles on this trip. You have targeted 8 potential cities. Some cities have more client or target market companies than others, so they will be weighted higher on your list of priorities.  In fact, you may even want to visit some cities more than one day since the number of potential visits is larger. 

In table, I have chosen to start at Albany, and to travel to up to 7 other cities. I have assigned a weighting to each city, from 1 (low priority) to 4 (high priority). I want to take up to 9 half-day trips and city visits, and I will then fly back home from one of these cities. 

The rules of my program are as follows:
The cities are each represented by a number (1=Albany, NY; 2= Augusta, ME; … 8= Montpelier, VT) and in total, I want to drive no more than 1,000 miles in 9 segments. 
I’ll give myself a credit for each city visited, based on its weight and the times I visited it. The first visit will be for the full weight, the second visit is only worth half that weight, and the third (or later) visit is worth only a tenth of the weight.  

In order to ensure that I don’t choose a route that exceeds a total of 1,000 miles, I’ll subtract 1 from my score for every mile that exceeds 1,000. Write code in python using a genetic algorithm to solve this traveling salesman problem and find the optimal route.

In [254]:
# imports
import numpy as np
from random import randint
import pandas as pd
import matplotlib.pyplot as plt

In [255]:
# Paramters
EPOCHS = 100
SETS_PER_GENERATION = 100
ELITES = 20
PARENTS_PER_CHILD = 50
MAX_MUTATIONS = 9
NUMBER_MUTATIONS = 3
CROSSOVER_VALUE = 2

In [256]:
cities = {'Albany':0, 'Augusta':1, 'Boston':2, 'Bridgeport':3, 'Concord':4, 'Greenwich':5, 'Hartford':6, 'Montpelier':7}
city_names = np.array(list(cities.keys())) 
number_of_cities = cities['Montpelier']+1
distances = {
    'Albany':[1,0,326,170,156,142,154,111,167],
    'Augusta':[2,326,0,164,310,153,339,256,192],
    'Boston':[3,170,164,0,153,68,182,99,179],
    'Bridgeport':[4,156,310,153,0,198,29,56,250],
    'Concord':[5,142,153,68,198,0,227,144,114],
    'Greenwich':[6,154,339,182,29,227,0,85,279],
    'Hartford':[7,111,256,99,56,144,85,0,194],
    'Montpelier':[8,167,192,179,250,114,279,194,0]
}

weights = [1,1,2,2,1,2,5,1]



The steps of the genetic algorithm
1. Initialize the population randomly.
2. Determine the fitness of the chromosome.
3. Until done repeat:
    1. Select parents.
    2. Perform crossover and mutation.
    3. Calculate the fitness of the new population.
    4. Append it to the gene pool.

In [257]:
# helper functions
def distance(idx_depart_city, idx_dest_city):
    depart_city = city_names[idx_depart_city]
    dest_city = city_names[idx_dest_city]
    distance = distances[depart_city][idx_dest_city]
    return distance


In [258]:
# First step: Create the first population set
def create_population(city_names):

    population_set = []
    for i in range(SETS_PER_GENERATION):
        #Randomly generating a new solution
        solution = np.random.randint(0, number_of_cities, 8)
        population_set.append(solution)
    return np.array(population_set)

In [259]:
population_set = create_population(city_names)
df = pd.DataFrame(population_set, columns=city_names)
df

,Albany,Augusta,Boston,Bridgeport,Concord,Greenwich,Hartford,Montpelier
0,5,7,6,0,3,3,2,0
1,2,5,1,2,1,2,1,1
2,1,0,4,3,6,2,5,3
3,7,6,3,7,4,2,2,1
4,5,2,0,4,6,2,7,3
...,...,...,...,...,...,...,...,...
95,7,4,1,3,2,7,4,6
96,3,2,0,5,4,5,2,3
97,0,0,0,0,0,3,3,5
98,4,7,5,0,2,1,3,1


In [268]:
# Caluclate the total distance per solution

def initial_population_distance(population, weights):
    total = 0
    visited = []
    for i in range(number_of_cities-1):
        a = population[i]
        b = population[i+1]
        #print('The current destination city is: ' + str(b))
        d = distance(a,b)
        
        if b in visited:
            visit_count = visited.count(b)
            #print('The current visit_count is: ' + str(visit_count))
            if visit_count == 1:
                w = 0.5 * weights[population[i+1]]  # If visited once already
            elif visit_count == 2: 
                w = 0.1 * weights[population[i+1]]  # If visited twice
            else:
                w = 0 # If visited more than twice
            #print('The weight for the repeated city is: ' + str(w))
        else:
            w = weights[b]
        #print('The weight is: ' + str(w))
        e = d * w
        #print(d,e)
        total += e
        
        if total > 1000:
            total = total - (total-1000)
            
        visited.append(b)
        #print(visited)
    return total


In [270]:
weights = [1,1,2,2,1,2,5,1]
population = [1, 3, 7, 1, 2, 0, 4, 0]

print(initial_population_distance(population, weights))


712.5


In [271]:
# Calculate the weight for all solutions and add them to the dataframe
scores = []
for index, row in df.iterrows():
    population = row.values
    score = initial_population_distance(population, weights)
    scores.append(score)

df['Score'] = scores  # Add the scores as a new column in your DataFrame

In [272]:
df

,Albany,Augusta,Boston,Bridgeport,Concord,Greenwich,Hartford,Montpelier,Score
0,5,7,6,0,3,3,2,0,1000.0
1,2,5,1,2,1,2,1,1,392.0
2,1,0,4,3,6,2,5,3,1000.0
3,7,6,3,7,4,2,2,1,1000.0
4,5,2,0,4,6,2,7,3,1000.0
...,...,...,...,...,...,...,...,...,...
95,7,4,1,3,2,7,4,6,1000.0
96,3,2,0,5,4,5,2,3,1000.0
97,0,0,0,0,0,3,3,5,890.6
98,4,7,5,0,2,1,3,1,1000.0


In [273]:
# sort the dataframe in descending order of the score col
sorted_df = df.sort_values(by='Score', ascending=False)
sorted_df

,Albany,Augusta,Boston,Bridgeport,Concord,Greenwich,Hartford,Montpelier,Score
0,5,7,6,0,3,3,2,0,1000.0
38,6,4,1,4,1,5,1,3,1000.0
64,4,3,7,4,4,1,7,5,1000.0
61,5,5,0,4,0,1,1,3,1000.0
60,3,6,7,5,1,4,0,6,1000.0
...,...,...,...,...,...,...,...,...,...
45,3,7,4,3,7,0,4,0,558.5
57,7,0,7,0,5,4,4,4,554.8
91,4,5,4,5,4,2,7,0,456.5
72,2,1,0,1,0,7,0,4,440.8


In [274]:
# Function to perform crossover
def crossover(parent1, parent2):
    crossover_point = len(parent1) // CROSSOVER_VALUE
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child


In [275]:
# Function to perform mutation
def mutation(row):
    indices_to_mutate = np.random.choice(len(row), NUMBER_MUTATIONS, replace=False)
    row[indices_to_mutate] = 1 - row[indices_to_mutate]  # Flip 0s to 1s and vice versa
    return row


In [276]:
# rewrite the population_distance function to allow for it to take a df row as its input
def population_distance(row, distances, weights):
    total = 0
    visited = set()
    cities = row.values[:-1]  # Exclude the last column ('Score')

    for i in range(len(cities) - 1):
        a = city_names[i]
        b = cities[i + 1]
        d = distances[a][b]  # Assuming distances is a matrix or dictionary

        if b in visited:
            visit_count = list(cities).count(b)
            if visit_count == 1:
                w = 0.5 * weights[b]  # If visited once already
            elif visit_count == 2:
                w = 0.1 * weights[b]  # If visited twice
            else:
                w = 0  # If visited more than twice
        else:
            w = weights[b]

        e = d * w
        total += e

        if total > 1000:
            total -= (total - 1000)

        visited.add(b)

    return total


In [279]:
# create the new dataframes with the mutations and crossover
def evolve_population(sorted_df):
    elites = sorted_df.head(ELITES)  # Top 20 rows as elites
    children = []
    # weights = [1,1,2,2,1,2,5,1]

    # Select next 50 rows and perform crossover to create 80 children
    next_50 = sorted_df.iloc[20:70]
    for i in range(80):
        random_rows = next_50.sample(n=2)
        child = crossover(random_rows.iloc[0].values, random_rows.iloc[1].values)
        children.append(child)

    # Exclude 'Score' column from children_df creation
    children_df = pd.DataFrame(children, columns=sorted_df.columns)  # Include all columns

    # Randomly select 9 rows and perform mutation
    mutated_indices = np.random.choice(len(children), MAX_MUTATIONS, replace=False)
    for index in mutated_indices:
        children_df.iloc[index] = mutation(children_df.iloc[index])

    # Combine elites and mutated children to form the final population
    final_population = pd.concat([elites, children_df.iloc[:len(elites)]])
    final_population = final_population.drop('Score', axis=1)
    final_population = final_population.astype(int)
    

    # calculate the score for each row and sort by Score
    scores = []
    for index, row in final_population.iterrows():
        score = population_distance(row, distances, weights)
        scores.append(score)
    final_population['Score'] = scores
    final_population = final_population.sort_values(by='Score')

    return final_population

In [281]:
df = evolve_population(sorted_df)
df

,Albany,Augusta,Boston,Bridgeport,Concord,Greenwich,Hartford,Montpelier,Score
52,3,1,5,3,5,0,6,7,350.6
3,2,1,1,5,5,7,7,5,360.7
38,6,4,1,4,1,5,1,3,497.3
61,5,5,0,4,0,1,1,3,596.8
47,1,7,1,4,7,0,4,3,603.5
18,1,-6,7,0,7,0,4,-2,634.1
14,3,4,7,4,6,3,0,1,714.3
2,3,1,4,5,3,3,4,2,754.9
0,4,4,1,5,6,4,4,6,763.0
9,6,1,4,5,3,3,5,5,811.0


In [285]:
evolved_populations = []  # Initialize the list
def run_evolution():
    for _ in range(EPOCHS):
            evolved_population = evolve_population(sorted_df)
            evolved_populations.append(evolved_population)
            evolved_population_df = pd.concat(evolved_populations, ignore_index=True)
            return evolved_population_df

# Call the function to run evolution 100 times
results = run_evolution()

In [286]:
results


,Albany,Augusta,Boston,Bridgeport,Concord,Greenwich,Hartford,Montpelier,Score
0,3,1,5,3,5,0,6,7,350.6
1,2,1,1,5,6,3,1,5,417.0
2,1,0,4,3,6,4,4,6,456.0
3,4,0,4,1,0,0,1,5,496.4
4,6,4,1,4,1,5,1,3,497.3
5,7,4,1,3,1,1,3,5,518.4
6,5,5,0,4,0,1,1,3,596.8
7,1,7,1,4,7,0,4,3,603.5
8,4,5,2,4,2,4,1,3,653.0
9,2,4,4,7,5,5,6,5,682.0
